# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm: Romberg Integration

In [1]:
import time

In [2]:
import numpy as np
import sys

np.set_printoptions(precision=14, linewidth=120)

# Algorithm

In [3]:
%%time
def integrate(fn, a, b, steps=5, debug=False, exact=None):
    table = np.zeros((steps, steps), dtype=np.float64)
    pow_4 = 4 ** np.arange(steps, dtype=np.float64) - 1

    # trapezoidal rule
    h = (b - a)
    table[0, 0] = h * (fn(a) + fn(b)) / 2

    for j in range(1, steps):
        h /= 2

        # extended trapezoidal rule
        table[j, 0] = table[j - 1, 0] / 2
        table[j, 0] += h * np.sum(fn(a + i * h) for i in range(1, 2 ** j + 1, 2))

        # richardson extrapolation
        for k in range(1, j + 1):
            table[j, k] = table[j, k - 1] + (table[j, k - 1] - table[j - 1, k - 1]) / pow_4[k]

    # debug
    if debug:
        print(table, file=sys.stderr)
        if exact is not None:
            errors = ['%.2e' % i for i in np.abs(table.diagonal() - exact)]
            print('abs. error:', errors, file=sys.stderr)

    return table[-1, -1]

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 11.9 µs


# Run

## Integration

In [4]:
%%time
# integral[0, 1] of e^(-x^2)
integrate(lambda x: np.exp(-x * x), 0, 1, debug=True, exact=0.746824132812427)

CPU times: user 3.31 ms, sys: 853 µs, total: 4.16 ms
Wall time: 4.32 ms


<timed exec>:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
[[0.68393972058572 0.               0.               0.               0.              ]
 [0.73137025182856 0.74718042890951 0.               0.               0.              ]
 [0.74298409780038 0.74685537979099 0.74683370984975 0.               0.              ]
 [0.7458656148457  0.74682612052747 0.7468241699099  0.74682401848228 0.              ]
 [0.74658459678822 0.74682425743573 0.74682413322961 0.74682413264739 0.74682413309509]]
abs. error: ['6.29e-02', '3.56e-04', '9.58e-06', '1.14e-07', '2.83e-10']


0.7468241330950943

In [5]:
%%time
# ln(2)
integrate(1..__truediv__, 1, 2, debug=True, exact=np.log(2))

CPU times: user 4.16 ms, sys: 27 µs, total: 4.19 ms
Wall time: 4.31 ms


<timed exec>:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
[[0.75             0.               0.               0.               0.              ]
 [0.70833333333333 0.69444444444444 0.               0.               0.              ]
 [0.69702380952381 0.69325396825397 0.6931746031746  0.               0.              ]
 [0.69412185037185 0.69315453065453 0.69314790148123 0.69314747764483 0.              ]
 [0.69339120220753 0.69314765281942 0.69314719429708 0.69314718307193 0.69314718191674]]
abs. error: ['5.69e-02', '1.30e-03', '2.74e-05', '2.97e-07', '1.36e-09']


0.693147181916745

In [6]:
%%time
# integral[0, 1] of x^3
integrate(lambda x: x**3, 0, 1, debug=True, exact=.25)

CPU times: user 3.17 ms, sys: 0 ns, total: 3.17 ms
Wall time: 3.53 ms


<timed exec>:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
[[0.5          0.           0.           0.           0.          ]
 [0.3125       0.25         0.           0.           0.          ]
 [0.265625     0.25         0.25         0.           0.          ]
 [0.25390625   0.25         0.25         0.25         0.          ]
 [0.2509765625 0.25         0.25         0.25         0.25        ]]
abs. error: ['2.50e-01', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']


0.25

## Logarithmus Naturalis

In [7]:
%%time
def ln(x):
    if x <= 0:
        raise ValueError()
    m, e = np.frexp(x)
    return integrate(1..__truediv__, 1, m) + e * 0.6931471805599453

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.78 µs


In [8]:
%%time
ln(np.e)

CPU times: user 393 µs, sys: 0 ns, total: 393 µs
Wall time: 411 µs


<timed exec>:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


0.999999999997895

In [9]:
%%time
ln(2)

CPU times: user 326 µs, sys: 0 ns, total: 326 µs
Wall time: 334 µs


<timed exec>:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


0.6931471792031456

In [10]:
%%time
ln(np.pi)

CPU times: user 346 µs, sys: 0 ns, total: 346 µs
Wall time: 353 µs


<timed exec>:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


1.1447298858493882

## Normal Distribution

In [11]:
%%time
def norm_pdf(x, mean, sd):
    x0 = (x - mean) ** 2
    v2 = 2 * sd ** 2
    return np.exp(-x0 / v2) / np.sqrt(np.pi * v2)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


In [12]:
%%time
def norm_cdf(x, mean, sd):
    return integrate(lambda x: norm_pdf(x, mean, sd), mean, x) + .5

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.34 µs


In [13]:
%%time
norm_cdf(1.96, mean=0, sd=1)

CPU times: user 436 µs, sys: 0 ns, total: 436 µs
Wall time: 461 µs


<timed exec>:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


0.9750021118942748

In [14]:
%%time
norm_cdf(1.2, mean=1, sd=.2) - norm_cdf(0.8, mean=1, sd=.2)

CPU times: user 470 µs, sys: 95 µs, total: 565 µs
Wall time: 573 µs


<timed exec>:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


0.6826894921375496

## The End